In [1]:
%pip install pillow==11.2.1
%pip install tqdm==4.67.1

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import cv2
from tqdm import tqdm
from PIL import Image, ImageOps, ImageEnhance

In [3]:
os.getcwd()

'c:\\Users\\H12838\\Desktop\\dataset'

In [4]:
prefix = os.path.abspath(r'')
aug_source_root = rf"{prefix}\img_cropped_restored"
aug_output_root = rf"{prefix}\img_cropped_augmented"
restored_input_root = rf"{prefix}\img_cropped"                # 裁剪错色的图像目录
restored_output_root = rf"{prefix}\img_cropped_restored"      # 恢复后输出目录

In [5]:
w = 100 # 要resize的图片的宽度
h = 100 # 要resize的图片的高度

In [6]:
def safe_open_image(path):
    try:
        return Image.open(path).convert("RGB")
    except Exception as e:
        print(f"❌ 无法打开 {path}: {e}")
        return None
    
    # 不反色，只做亮度 + 对比度 + 翻转
def natural_augment(img):
    img = ImageEnhance.Brightness(img).enhance(1.2)     # 稍微提亮
    img = ImageEnhance.Contrast(img).enhance(1.3)       # 增强边缘
    img = ImageOps.mirror(img)                          # 水平翻转
    return img

In [13]:
def image_resizer(source_path, save_path, height, width): # height: 垂直高度，width: 水平寬度，(height, width) = 圖片處理後的大小

    image = cv2.imread(source_path)
    size = image.shape # 原圖的尺寸
    h = size[0]
    w = size[1]

    if h / w > 1: # 高度大於寬度
        rate = height / h
    elif w / h == 1: # 正方形
        rate = height / h
    elif 0 < h / w < 1: # 高度小於寬度
        rate = width / h
    else:
        print(f'錯誤')

    # print(f'{source_path}\n({w}, {h}) -> ({w * rate}, {h * rate})')
    if rate > 1:
        new_image = cv2.resize(image, (round(w * rate), round(h * rate)), interpolation=cv2.INTER_CUBIC)
        if h / w > 1:
            save_image = cv2.copyMakeBorder(new_image, 0, 0, 0, width - round(w * rate), cv2.BORDER_CONSTANT,value=[0,0,0])
        elif h / w ==1:
            save_image = new_image
        elif 0 < h / w < 1:
            save_image = cv2.copyMakeBorder(new_image, 0, height - round(h * rate), 0, 0, cv2.BORDER_CONSTANT,value=[0,0,0])

        cv2.imwrite(save_path, save_image)

    elif rate < 1:
        new_image = cv2.resize(image, (round(w * rate), round(h*rate)), interpolation=cv2.INTER_AREA)
        if h / w > 1:
            save_image = cv2.copyMakeBorder(new_image, 0, 0, 0, width - round(w * rate), cv2.BORDER_CONSTANT,value=[0,0,0])

        elif h / w ==1:
            save_image = new_image

        elif 0 < h / w < 1:
            save_image = cv2.copyMakeBorder(new_image, 0, height - round(h * rate), 0, 0, cv2.BORDER_CONSTANT,value=[0,0,0])

        cv2.imwrite(save_path, save_image)

#### 反色恢复

In [8]:
os.makedirs(restored_output_root, exist_ok=True)
for emotion in os.listdir(restored_input_root):
    src_dir = os.path.join(restored_input_root, emotion)
    dst_dir = os.path.join(restored_output_root, emotion)
    os.makedirs(dst_dir, exist_ok=True)
    for fname in tqdm(os.listdir(src_dir), desc=f"Restoring {emotion}"):
        img_path = os.path.join(src_dir, fname)
        img = cv2.imread(img_path)
        if img is None:
            continue
        restored = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)  # 反转颜色通道
        cv2.imwrite(os.path.join(dst_dir, fname), restored)


Restoring surprise: 100%|██████████| 8788/8788 [00:04<00:00, 2041.97it/s]


#### 数据增强 + resize

In [9]:
os.makedirs(aug_output_root, exist_ok=True)
for emotion in os.listdir(aug_source_root):
    os.makedirs(os.path.join(aug_output_root, emotion), exist_ok=True)

In [14]:
for emotion in os.listdir(aug_source_root):
    src_dir = os.path.join(aug_source_root, emotion)
    dst_dir = os.path.join(aug_output_root, emotion)
    for fname in tqdm(os.listdir(src_dir), desc=f"Processing {emotion}"):
        fpath = os.path.join(src_dir, fname)
        img = safe_open_image(fpath)
        if img is None:
            continue
        aug = natural_augment(img)
        base, ext = os.path.splitext(fname)
        aug.save(os.path.join(dst_dir, f"{base}_aug{ext}"))
        image_resizer(
    os.path.join(dst_dir, f"{base}_aug{ext}"),
    os.path.join(dst_dir, f"{base}_aug{ext}"),
    h,
    w
)

Processing surprise: 100%|██████████| 8788/8788 [32:17<00:00,  4.54it/s]   
